# Predicting the stock market 

In this project, we are going to predict a closing value of S&P500 index. The data set can be downloaded from [here](https://www.kaggle.com/samaxtech/sp500-index-data). This index is often considered as one of good representations of the U.S. stock market. 
As a note, This is a basic machine learning project and this should not be used for any purposes in practice. 

In [3]:
import pandas as pd 
import numpy as np
from datetime import datetime

## Read the data set 

In [4]:
index_price = pd.read_csv('sphist.csv') 

In [5]:
index_price.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


In [6]:
index_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       16590 non-null  object 
 1   Open       16590 non-null  float64
 2   High       16590 non-null  float64
 3   Low        16590 non-null  float64
 4   Close      16590 non-null  float64
 5   Volume     16590 non-null  float64
 6   Adj Close  16590 non-null  float64
dtypes: float64(6), object(1)
memory usage: 907.4+ KB


Since ```Date``` column is object type, transform it into datetime. It seems like there is no null values.

In [7]:
index_price['Date'] = pd.to_datetime(index_price['Date'])

In [8]:
index_price.sort_values(by = 'Date', inplace=True)

In [9]:
index_price.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


## Creating indicators

I will create three indicators for each row. 
Those indicators are like these....
- The average closing price from the past 5 days.
- The average closing price for the past 30 days.
- The average closing price for the past 365 days.

### AVG price of the past 5 days 

In [10]:
rolling_close = index_price['Close'].rolling(5).mean()

In [11]:
rolling_close.head(10)

16589       NaN
16588       NaN
16587       NaN
16586       NaN
16585    16.900
16584    16.974
16583    17.022
16582    16.988
16581    16.926
16580    16.854
Name: Close, dtype: float64

Here is an important step. I cannot combine this series with our data set since doing so will result in adding future data to each row. Thus, I have to shift the series by 1 period.

In [12]:
shifted_rolling_5 = rolling_close.shift(1)

In [13]:
shifted_rolling_5.name = 'day5'

In [14]:
shifted_rolling_5.head(10)

16589       NaN
16588       NaN
16587       NaN
16586       NaN
16585       NaN
16584    16.900
16583    16.974
16582    17.022
16581    16.988
16580    16.926
Name: day5, dtype: float64

### AVG prices of the past 30 and 365 days. 

In [15]:
other_indicators = []
rollings = [30,365]
for num in rollings:
    rolling_close = index_price['Close'].rolling(num).mean()
    shifted_rolling = rolling_close.shift(1)
    shifted_rolling.name = "day_{}".format(num)
    other_indicators.append(shifted_rolling)

In [16]:
indicators = pd.concat([shifted_rolling_5,*other_indicators],axis=1)

In [17]:
with_indicators = pd.concat([index_price,indicators],axis=1)

## Cleaning data

Some of rows now have missing values in indicators. They are not useful for machine learning models, so I will drop rows containing missing values.

In [18]:
with_indicators = with_indicators.dropna()

In [19]:
with_indicators

,Date,Open,High,Low,Close,Volume,Adj Close,day5,day_30,day_365
16224,1951-06-19,22.020000,22.020000,22.020000,22.020000,1.100000e+06,22.020000,21.800000,21.703333,19.447726
16223,1951-06-20,21.910000,21.910000,21.910000,21.910000,1.120000e+06,21.910000,21.900000,21.683000,19.462411
16222,1951-06-21,21.780001,21.780001,21.780001,21.780001,1.100000e+06,21.780001,21.972000,21.659667,19.476274
16221,1951-06-22,21.549999,21.549999,21.549999,21.549999,1.340000e+06,21.549999,21.960000,21.631000,19.489562
16220,1951-06-25,21.290001,21.290001,21.290001,21.290001,2.440000e+06,21.290001,21.862000,21.599000,19.502082
...,...,...,...,...,...,...,...,...,...,...
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883,2087.024023,2073.984998,2035.531178
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010,2090.231982,2076.283993,2035.914082
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117,2088.306006,2077.908659,2036.234356
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941,2080.456006,2078.931331,2036.507343


Some of columns contain future information, so I have to drop them. 
Those columns are...
- ```Open``` 
- ```High```
-```Low```
-```Volume```
-```Adj Close```

In [20]:
cleaned = with_indicators.drop(['Open','High','Low','Volume','Adj Close'],axis=1)

In [21]:
cleaned.head()

,Date,Close,day5,day_30,day_365
16224,1951-06-19,22.020000,21.800,21.703333,19.447726
16223,1951-06-20,21.910000,21.900,21.683000,19.462411
16222,1951-06-21,21.780001,21.972,21.659667,19.476274
16221,1951-06-22,21.549999,21.960,21.631000,19.489562
16220,1951-06-25,21.290001,21.862,21.599000,19.502082


## Linear regression(1st trial)

I have to choose error metric first. Given in a context of predicting stock markets, I think that predicting a significantly different situation could cause a large loss of money. Thus, I want to penilize large deviations between my predictions and actual values. Thus, I am going to use RMSE as an error metric. 

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [23]:
# separating a dateset into test and training dataset
cutoff = "2013-01-01"
train = cleaned.loc[cleaned['Date']<=cutoff,:]
test = cleaned.loc[cleaned['Date']>cutoff,:]
train = train[train.columns.drop('Date')]
test = test[test.columns.drop('Date')]

In [24]:
train.tail()

,Close,day5,day_30,day_365
743,1426.660034,1437.360010,1405.926001,1326.114028
742,1419.829956,1436.620019,1407.486336,1326.412494
741,1418.099976,1431.228003,1408.813000,1326.716494
740,1402.430054,1427.685986,1410.265332,1326.995836
739,1426.189941,1419.434009,1411.830001,1327.261562


In [25]:
test.head()

,Close,day5,day_30,day_365
738,1462.420044,1418.641992,1414.258667,1327.534055
737,1459.369995,1425.793994,1417.676668,1327.908247
736,1466.469971,1433.702002,1420.092668,1328.224877
735,1461.890015,1443.376001,1422.714665,1328.557617
734,1457.150024,1455.267993,1425.076664,1328.898603


In [26]:
features = train.columns.drop('Close')
target = train['Close']

In [27]:
lr = LinearRegression()
lr.fit(train[features],target)
predictions = lr.predict(test[features])

In [28]:
mean_squared_error(test['Close'],predictions,squared=False)

22.22006532421962

Okay. it seems that my predictions are off by 22.22 on average. Let's add more features and see if I can get better

## Adding more features

I will add three more features. 
- average volume over the past five days 
- standard deviation of volume over the past five days. 

In [29]:
index_price.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


In [30]:
#creating a feature describing average volume over the past five days 
rolling_avgv = index_price['Volume'].rolling(5).mean()
shifted_avgv=rolling_avgv.shift(1)

In [31]:
# creating a feature describing standard deviation of volume over the past five days 
rolling_sd = index_price['Volume'].rolling(5).std()
shifted_sd = rolling_sd.shift(1)

In [32]:
shifted_avgv.name = 'avgv_5'
shifted_avgv.head(6)

16589          NaN
16588          NaN
16587          NaN
16586          NaN
16585          NaN
16584    2046000.0
Name: avgv_5, dtype: float64

In [33]:
shifted_sd.name = 'sdV_5'
shifted_sd.head(6)

16589              NaN
16588              NaN
16587              NaN
16586              NaN
16585              NaN
16584    529650.828377
Name: sdV_5, dtype: float64

In [34]:
with_indicators_2 = pd.concat([index_price,shifted_rolling_5,*other_indicators,shifted_avgv,shifted_sd],axis=1)

In [35]:
with_indicators_2.head(6)

,Date,Open,High,Low,Close,Volume,Adj Close,day5,day_30,day_365,avgv_5,sdV_5
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN,NaN,NaN
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN,NaN,NaN
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN,NaN,NaN
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN,NaN,NaN
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN,NaN,NaN,NaN,NaN
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.9,NaN,NaN,2046000.0,529650.828377


In [36]:
with_indicators_2 = with_indicators_2.dropna()

In [37]:
cleaned_2 = with_indicators_2.drop(['Open','High','Low','Volume','Adj Close'],axis=1)

In [38]:
cleaned_2.head()

,Date,Close,day5,day_30,day_365,avgv_5,sdV_5
16224,1951-06-19,22.020000,21.800,21.703333,19.447726,1196000.0,142232.204511
16223,1951-06-20,21.910000,21.900,21.683000,19.462411,1176000.0,148425.065269
16222,1951-06-21,21.780001,21.972,21.659667,19.476274,1188000.0,138816.425541
16221,1951-06-22,21.549999,21.960,21.631000,19.489562,1148000.0,126767.503722
16220,1951-06-25,21.290001,21.862,21.599000,19.502082,1142000.0,113666.177907


In [39]:
cutoff = "2013-01-01"
train = cleaned_2.loc[cleaned_2['Date']<=cutoff,:]
test = cleaned_2.loc[cleaned_2['Date']>cutoff,:]
train = train[train.columns.drop('Date')]
test = test[test.columns.drop('Date')]

## Linear regression (2nd trial)

In [40]:
features = train.columns.drop('Close')
target = train['Close']

In [41]:
lr = LinearRegression()
lr.fit(train[features],target)
predictions = lr.predict(test[features])

In [42]:
mean_squared_error(test['Close'],predictions,squared=False)

22.28188273332705

It seems that my model don't get improved. Thus, it is better to remove the new indicators and create new other features. There could be ways to improve the model more. However, for now, I will stop here. 